# CLASIFICADOR y SIFT

Como SIFT se trata de una ampliación de CLASIFICADOR, se ha decidido incluir ambos en un mismo notebook.

# Índice

[Enunciados](#Enunciados)

[Ficheros](#Ficheros)

[Código](#Código)

- [clasificadorConstructor.py](#clasificadorConstructor.py)
- [clasificadorColeccion.py](#clasificadorColeccion.py)
- [clasificador.py](#clasificador.py)

[Conceptos teóricos](#Conceptos-teóricos)

[Ejecución del código: Ejemplo de funcionamiento](#Ejecución-del-código:-Ejemplo-de-funcionamiento)

[Tardanza del código](#Tardanza-del-código)

## Enunciados

**Enunciado de CLASIFICADOR:**

Prepara una aplicación sencilla de reconocimiento de imágenes. Debe admitir (al menos) dos argumentos:

- `--models=<directorio>`, la carpeta donde hemos guardado un conjunto de imágenes de objetos o escenas que queremos reconocer.

- `--method=<nombre>`, el nombre de un método de comparación.

Cada fotograma de entrada se compara con los modelos utilizando el método seleccionado y se muestra información sobre el resultado (el modelo más parecido o probable, las distancias a los diferentes modelos, alguna medida de confianza, etc.). Implementa inicialmente un método basado en el "embedding" obtenido por [mediapipe](https://developers.google.com/mediapipe/solutions/vision/image_embedder) (`code/DL/embbeder`).

**Enunciado de SIFT:**

Añade al ejercicio CLASIFICADOR un método basado en el número de coincidencias de `keypoints` SIFT. Utilízalo para reconocer objetos con bastante textura (p. ej. carátulas de CD, portadas de libros, cuadros de pintores, etc.).

## Ficheros

- Carpeta `code`: Aquí se encuentran los ficheros de código
- Carpeta `images`: Aquí se encuentran imágenes de prueba para el clasificador.
- Carpeta `imports`: Aquí se encuentran los ficheros importantes para el funcionamiento del programa.

## Código

### clasificadorConstructor.py:

En este fichero se encuentra la clase `Clasificador` cuyas subclases son los distintos métodos de clasificación que se pueden aplicar. 

Se crea un clasificador por cada imagen de ``models``, debido a que esto permite que cada instancia de subclase guarde transformaciones a la imagen en la inicialización de la instancia, y se haga una sola vez, en lugar de una vez por cada frame.

De esta forma, en el método de inicialización (`\_\_init\_\_`), se pasa como parámetro el path a la imagen ("imgPath") y  el nombre de la imagen ("nombreImg"). Se guarda el nombre de la imagen, de forma que se pueda llamar al método `getNombreImg()` cada vez que este se necesite. 

Cada subclase tiene un método estático llamado \textit{getMethod()} que devuelve el nombre del clasificador. Este es el nombre con el que se llama a ese clasificador concreto (`--method`) en los parámetros de entrada del programa. 

El método de clase `changeFrame` recibe como parámetro el frame actual. De esta forma, este método permite realizar las transformaciones necesarias al frame y guardarlas en variables de clase.

Así, en lugar de hacer las transformaciones una vez por cada instancia, se hacen las transformaciones una vez por frame.

El método llamado `similarity` devuelve la similaridad del frame actual (el que se guardó la última vez que se llamó a `changeFrame()`), con la imagen de la instancia. También devuelve el frame modificado según se quiera. Por ejemplo, en el método `skimageHog` se dibuja un rectángulo en el frame en la posición con mayor similaridad.

### clasificadorColeccion.py:

Contiene las subclases de la clase Clasificador. Cada subclase implementa \textit{similarity}, \textit{changeFrame}, y el constructor de la subclase. A su vez, cada una implementa el método `getMethod()` en el que devuelven el nombre del método.

- **embedder**

  Se inicializa la clase con el método `\_\_init\_\_`. Este método llama a un método de clase (`inicializoClase`) que crea un objeto `ImageEmbedder` con el modelo almacenado en `./embedder.tflite`. También se utiliza la imagen del clasificador para guardar su embedding (calculado con el modelo).

  Es importante indicar que sólo se llamará a `inicializoClase` una vez (y no en todas las instancias), de forma que se comparte el modelo entre todas las instancias de esta subclase.

  El método `changeFrame` guarda el frame pasado como parámetro y su embedding (calculado con el modelo).

  En el método `similarity`, se utiliza un método de mediapipe que calcular la similaridad entre los embeddings del frame y de la imagen.

  El frame se modifica mostrando la imagen y la similaridad obtenida, además del frame.

- **skimageHog**

  Se inicializa la clase con el método `\_\_init\_\_`, donde se guarda el nombre de la imagen de la clase, el histograma de orientaciones del gradiente (HOG) de esta imagen, la altura y anchura del HOG y el HOG aplanado (con el uso del método `flatten()`). 

  Dado que estas operaciones son muy lentas, guardar el HOG, y este aplanado, ayuda a no tener que calcularlo por cada frame, y por lo tanto que la aplicación resulte más rápida.

  En el método `changeFrame` se guarda el HOG del frame, su altura y su anchura. No se guarda este aplanado porque se debe calcular en similarity, cómo se verá a continuación.

  En el método `similarity` se calcula la distancia entre los HOGs de la imagen de la instancia y el frame almacenado.

  La distancia entre HOGs de dos imágenes se calcula entre HOGs de igual tamaño, por lo que se calcula esta distancia entre el HOG de la imagen y cada una de las posibles partes del HOG del frame que tengan el mismo tamaño que el de la imagen. 

  Esto se hace mediante dos bucles que van desde el principio del HOG del frame (0,0) hasta la diferencia de altura o anchura de los HOGs (si se pasa de la diferencia, no se puede comparar con la imagen porque faltarían valores a la derecha/debajo del HOG del frame).

  Esta distancia se calcula entre HOGs aplanados, de forma que resulte más cómoda su comparación. Una vez aplanado, bastaría con ir por cada bloque y sumar la diferencia de cada par de bloques del HOG, entre el número de bloques totales. Esta diferencia se calcula mirando la diferencia de la magnitud de los vectores de cada orientación.

  De estas distancias, se busca la más pequeña, y se devuelve 1 - la distancia, de forma que cuanto más grande sea el valor, más parecidas son las imágenes.

  Por esto el frame no se guarda aplanado, ya que se debe aplanar cada parte del HOG del frame para compararla con la imagen. Por ello, no sirve el HOG entero aplanado y se debe aplanar cada vez.

  El frame en `similarity` se modifica, dibujando un rectángulo en la posición de la imagen con mayor similaridad. Se devuelve el frame modificado.

  A su vez, se dibuja la distancia entre el frame y la imagen, y la imagen de la instancia en la esquina superior izquierda del frame.

  Se devuelve este frame modificado, y 1 menos la distancia menor, de forma que cuanto más grande sea, más similares sean la imagen y la parte del frame seleccionada.

- **sift**

  Se inicializa la clase con el método `\_\_init\_\_`, que guarda el atributo "bf" que sirve para comparar los keypoints de dos imágenes y dar las dos mejores coincidencias para cada punto, y el atributo "sift" que sirve para detectar los keypoints de una imagen. Estos atributos se guardan en atributos de clase, para que todas las distintas instancias de la subclase las compartan.

  También se guardan los keypoints y los descriptores de estos de la imagen cuyo nombre se pasa como parámetro, de forma que no se tiene que calcular por cada frame.

  En el método `changeFrame` se utiliza "sift" para obtener los keypoints del frame pasado como parámetro, y sus descriptores.

  En el método `similarity`, se obtienen los dos mejores matches de cada keypoint con "bf". Estos matches se calculan con los keypoints (sus descriptores) de la imagen de la instancia con el frame almacenado.

  Posteriormente, se hace el test de ratio para quedarse solo con los mejores matches.

  El test de ratio se basa en quedarse con la mejor coincidencia de keypoints solo si hay mucha diferencia entre el parecido de la mejor con el keypoint, y el parecido de la segunda mejor con el keypoint. De esta forma, se evita escoger coincidencias erróneas.

  Después, se dibujan los matches en el frame y se escribe el número de keypoints que coincide. Se devuelve el frame modificado y la similaridad. 

  Para calcular la similaridad no se pueden devolver los keypoints que coinciden a secas, debido a que, si una imagen tuviese significativamente más keypoints que otra (por ejemplo 100 vs 10), es probable que termine teniendo más matches con el frame aunque el frame (o un trozo de este) se parezca más a la imagen con pocos keypoints. Para evitar esto, se divide el número de keypoints que coinciden entre el número de keypoints que tiene la imagen; de forma que se mira el porcentaje de keypoints de la imagen que se han encontrado en el frame.

### clasificador.py:
Es la clase principal del programa, es la que se ejecuta para que funcione el clasificador.

En primer lugar, se recogen los parámetros de entrada del programa (directorio de modelos y método de comparación). Se comprueba que se han pasado los dos parámetros necesarios, que no se han escrito parámetros no conocidos, y que el directorio de modelos existe.

Se obtiene el clasificador (de la clase Clasificador). Para ello, se recorren las subclases y se queda con la subclase cuyo atributo método coincide con el método pasado como parámetro. Si este no existe, se muestra un mensaje indicándolo y se cierra el programa.

Cuando se tiene la subclase, se crea una instancia de esta por cada imagen del directorio de modelos.

Se empieza a capturar el vídeo. Por cada frame que se captura, se llama al método `changeFrame` de la subclase escogida. Se llama al método `similarity` con todas las instancias de la subclase y se almacena el frame (modificado) devuelto por la instancia que devuelva la mayor similaridad. Por último, se muestra por pantalla el frame modificado.

  

## Conceptos teóricos

**Gradiente**:

El gradiente es un vector que indica hacia donde aumenta la luz. El gradiente de una imagen está compuesto por vectores que indican donde aumenta la luz por toda la imagen.

Cuando se trabaja con una imagen en blanco y negro, los cambios entre zonas blancas y zonas negras suelen ser zonas de bordes. Por ello, los cambios de gradiente suelen indicar la presencia de un borde. Al tratar con objetos, el fijarse en sus bordes realmente es fijarse en la estructura general de este.

Para utilizar el gradiente para clasificar objetos, en primer lugar se debe de suavizar la imagen para eliminar el ruido de la imagen que se toma del objeto y fijarse en el nivel justo de detalle, de forma que no se fije en los detalles pequeños sino en la estructura general de este.

Tal y como se ha dicho, el gradiente ayuda a reconocer objetos. Esto ocurre si el objeto es rígido, o tiene deformaciones pequeñas, de forma que una vez obtenido su gradiente este no cambia (o no lo hace de forma significativa). Para comparar dos fotografías y conocer si son el mismo objeto, en lugar comparar los gradientes, se utilizan los histogramas de orientaciones del gradiente.

**HOG**:

El HOG, también llamado el histograma de orientaciones del gradiente, es un conjunto de histogramas locales sobre las orientaciones discretizadas del gradiente.

Los vectores del gradiente, en lugar de representarlos con coordenadas x e y, se pueden representar de una forma polar con la magnitud del vector y su orientación. Esta forma de representarlos es muy útil, permitiendo la discretización de las orientaciones, dividiendolas en un cierto número (por ejemplo 16). En el caso de skimageHog, hay 8 orientaciones distintas.

Esta discretización de las orientaciones permite identificar objetos aunque hayan pequeños cambios.

Además, se crean histogramas locales de los vectores, de forma que se agrupan los vectores de píxeles cercanos y por cada agrupación se guardan las magnitudes totales de los vectores hacia cada orientación posible. Esta magnitud total en cada orientación se puede calcular de distintas formas. 

Una forma es sumar el número de vectores con esa orientación, pero esta forma no tiene en cuenta las diferencias de vectores, ya que no es lo mismo un vector que va de blanco a negro que un vector que va de un gris más claro a un gris más oscuro.

Una buena forma de calcular la magnitud total en cada orientación es sumando la magnitud de los vectores (locales) que están en esa orientación.

Una vez se tienen estos histogramas locales, se suelen normalizar. Esta normalización puede ser con sólo el histograma local. Sin embargo, es mejor normalizar cada histograma local teniendo en cuenta los histogramas locales cercanos a este.

De esta forma, se termina consiguiendo un HOG (conjunto de histogramas locales) que permite identificar objetos aunque hayan pequeños cambios en estos. Es importante recordar que esto se debe realizar sobre las fotografías en blanco y negro.

**SIFT**:

Este método calcula los ``keypoints'' de una imagen. Esto se refiere a puntos en una imagen que se diferencian de su entorno y, por lo tanto, el verlos en otra imagen hace que se puedan reconocer.

Por ejemplo, el borde de una mesa no es un buen punto debido a que este borde es igual al borde de la mesa un poco más a la izquierda. Sin embargo, las esquinas de la mesa sí son puntos clave, ya que no se parecen a su entorno. Por ello, es necesario escoger los puntos clave de la imagen. 

Esta búsqueda de puntos clave (keypoints) se calcula en el código del ejercicio con la función de OpenCV "detectAndCompute" aplicada a la imagen correspondiente, con el detector de puntos clave creado con la función "SIFT\_create()" de OpenCV.

A su vez, la función "knnMatch" aplicada a los descriptores de los puntos clave de dos imágenes calcula las mejores coincidencias de puntos clave entre las imágenes. Esto se hace con un comparador de puntos clave de OpenCV que se crea con la función "FMatcher".

Esto ayuda a comparar imágenes, de modo que cuantas más coincidencias de puntos clave obtengan, más se parecerán. Para ello, se debe hacer una limpieza de puntos, de manera que los puntos con cierta coincidencia pero con una coincidencia parecida con la segunda mejor coincidencia no se debe tener en cuenta, pero los que tienen una coincidencia muy distinta a la segunda mejor coincidencia se tienen en cuenta.

## Ejecución del código: Ejemplo de funcionamiento

Para ejecutar el código de CLASIFICADOR, se debe de ejecutar el código de `clasificador.py` desde la carpeta `CLASIFICADOR + SIFT` en el entorno de anaconda prompt explicado al inicio de la asignatura. Se deben de pasar como parámetros la carpeta en la que se encuentran las imágenes y el nombre del método. Si por ejemplo, se quiere realizar el método *Embedder* con la carpeta de fotografías que se encuentra ejecutando *cd ../images/ImagesEmbedder*, se debe de ejecutar el siguiente comando:

<p style="text-align: center;">clasificador.py --method=embedder --models=../images/ImagesEmbedder</p>

Es relevante mencionar que se muestra la imagen a la que más se parece el frame actual arriba a la izquierda.

- **Embedder**:

  Se van a presentar algunas imágenes del funcionamiento del clasificador Embedder. Si se utiliza una carpeta con imágenes  (models) de distintas zonas de una habitación (incluso personas) las identifica de zona correcta, aunque la imagen tenga distinta luminancia, resolución, o tamaño que los frames tomados con la cámara actual:

  <table style="width: 70%"><tr>
  <td> <img src="img/embedder-cara.png"/> </td>
  <td> <img src="img/embedder-otra-camara-luz.png"/> </td>
  <td> <img src="img/embedder-pared.png"/> </td>
  </tr></table>

  A su vez, funciona con cuadros, aunque estos se encuentren movidos o con colores algo alterados:

  <table style="width: 70%"><tr>
  <td> <img src="img/cuadro-2.png"/> </td>
  <td> <img src="img/cuadro-1.png"/> </td>
  <td> <img src="img/cuadros-error.png"/> </td>
  </tr></table>

  Como se ha podido observar, el clasificador falla en caso de que la fotografía se encuentre girada y más pequeña.

  Por lo tanto, para observar habitaciones funciona de forma correcta, pero con objetos (como cuadros) son preferibles otros clasificadores, a menos que estos objetos se muestren en la misma posición, distancia y ángulo que en la fotografía.

  Por último indicar que con objetos simples funciona de forma correcta a pesar de la diferencia de color entre la imagen y el frame, como se puede observar a continuación:

  <video src="img/objetos-simples-embedder.mp4" controls='play' style="width:40%">
  </video>

- **skimageHog**:

  En este filtro, hay que tener cuidado con qué objeto se escoge, ya que si se escoge un objeto muy simple, podría confundirse con otras cosas. Por ejemplo, en el siguiente vídeo, en vez de identificar las gafas todo el tiempo, en ocasiones se confunde con el fondo de la imagen al pensar que es un ratón:

  <video src="img/confunde-gafas-raton.mp4" controls='play' style="width:40%">
  </video>

- **sift**:

  El método SIFT funciona de forma correcta en cuadros y objetos complejos, pero hay que tener cuidado con los objetos simples, ya que, al tener pocos keypoints, son dificilmente identificables. Un ejemplo es el siguiente vídeo:

  <video src="img/objetos-simples-sift.mp4" controls='play' style="width:40%">
  </video>


## Tardanza del código

Para ver cuanto tarda en ejecutar el código se va a añadir las siguientes líneas de código al inicio del programa (antes de mostrar frames), y cuando se captura y cuando se muestra el frame:

In [ ]:
import time
# ...
inicio = time.time()

# Código que se hace cada frame

fin = time.time()
print(fin-inicio)

Como se puede observar, este código tarda 0 segundos porque no hace nada en medio, pero al añadirlo a `clasificador.py`, el tiempo aumenta:

Se obtiene:

**Inicio** (con 1 carpeta que contiene 4 imágenes de distintos cuadros):
- **Embedder**: *0.92* segundos
- **skimageHog**: *3.4* segundos
- **sift**: *5.8* segundos

**Cada frame** (con una carpeta que 4 imágenes de distintas zonas de una habitación):
- **Embedder**: Entre *0.01* y *0.02* segundos
- **skimageHog**: Entre *0.07* y *0.12* segundos
- **sift**: Entre *0.15* y *0.26* segundos

Se probó moviendo mucho la cámara, de ahí la diferencia de segundos en cada frame.

Como se ha podido obervar, *Embedder* es el método que menos tarda en ejecutarse, seguido por *skimageHog*, y por último *sift*.

Se tarda en iniciar debido a que se realizan tareas al inicio con las imágenes de forma que no se tengan que realizar por cada frame (y de este modo reducir el tiempo que tarda en mostrar cada frame y que vaya más fluido).

## Bibliografía

[MediaPipe Image Embedder](https://developers.google.com/mediapipe/solutions/vision/image_embedder/python)

[Comprobar la existencia de un fichero](https://www.geeksforgeeks.org/python-os-path-exists-method/)

[Comprobar la existencia de un directorio](https://www.geeksforgeeks.org/python-os-path-isdir-method/)

[Recorrer archivos de un directorio](https://www.codigopiton.com/como-listar-archivos-de-carpeta-en-python/)

[Material de la asignatura (código y notebooks)](https://github.com/albertoruiz/umucv/blob/master)